In [45]:
import pandas as pd

In [46]:
excel_dbfs_path = f"./david_finance.xlsx"
sheet_name = f"nueva_deuda_a_plazos"

In [47]:
dfp = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = sheet_name, dtype = {"monto_total_deuda":'float'})

In [48]:
tdc_df = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = "tarjetas_de_credito")

In [49]:
display(tdc_df.loc[tdc_df["payment_method"] == "LikeU"]["dia_de_corte"].values[0])
display(tdc_df.loc[tdc_df["payment_method"] == "LikeU"]["plazo_para_pagar"].values[0])
# si el dia del cargo es despues del corte sumamos el plazo para pagar 
# de lo contrario, es decir, si el dia de cargo es inferior al dia de corte, construimos el dia de pago sumando 1 mes al mes actual

12

19

In [83]:
class InstallmentsCreator:
    def __init__(self, df: pd.DataFrame):
        self.empty_installments_df = pd.DataFrame(
                        columns=['fecha_de_cargo' ,
                                'descripcion', 
                                'categoria', 
                                'monto', 
                                'forma_de_pago', 
                                'realizado_por', 
                                'limite_de_pago', 
                                'deuda_a_plazos_recurrente_o_normal', 
                                'comentarios'
                                ])

        self.tdc_df = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = "tarjetas_de_credito")
        self.condition_blank_comment = lambda text: text if text == text else ""
        self.df = df
        
        
    def get_pay_deadlines(self, row, date_range_i_row):
        try:
            dia_de_corte = self.tdc_df.loc[tdc_df["payment_method"] == row["forma_de_pago"]]["dia_de_corte"].values[0]
            plazo_para_pagar = self.tdc_df.loc[tdc_df["payment_method"] == row["forma_de_pago"]]["plazo_para_pagar"].values[0]
            return [
                pd.to_datetime(f"{date.year}-{date.month}-{dia_de_corte}", format = "%Y-%m-%d") + pd.DateOffset(days= plazo_para_pagar, months=1) 
                if date.day > dia_de_corte 
                else pd.to_datetime(f"{date.year}-{date.month}-{dia_de_corte}", format = "%Y-%m-%d") + pd.DateOffset(days= plazo_para_pagar) 
                for date in date_range_i_row
            ]
        except IndexError:
            return date_range_i_row


    def create_nthly_installments(self,row, date_range_i_row):
        return  pd.DataFrame(
            data={'fecha_de_cargo': date_range_i_row,
                    'descripcion': self.condition_blank_comment(row["descripcion"]), 
                    'categoria': row["categoria"], 
                    'monto': row["monto_total_deuda"] / row["numero_de_plazos"], 
                    'forma_de_pago': row["forma_de_pago"], 
                    'realizado_por': row["realizado_por"], 
                    'limite_de_pago': self.get_pay_deadlines(row, date_range_i_row), 
                    'deuda_a_plazos_recurrente_o_normal': "Plazos", 
                    'comentarios': self.condition_blank_comment(row["comentarios"])
                    }
                )

    
    def create_monthly_date_range(self, row):
        return  pd.date_range(
                    start = f"{row['fecha_de_inicio'].year}-{row['fecha_de_inicio'].month}-01", 
                    periods = row["numero_de_plazos"], 
                    freq = "MS"
                ) + pd.DateOffset(days=row['fecha_de_inicio'].day - 1)

    def create_monthly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "mensual"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_monthly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
    
        
    def create_semimonthly_date_range(self, row):
        return pd.date_range(
                    start = f"{row['fecha_de_inicio']}", 
                    periods = row["numero_de_plazos"], 
                    freq = "SME"
                ) + pd.DateOffset(days=row["dias_de_margen"])
    
    def create_semimonthly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "quincenal"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_semimonthly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def create_bimonthly_date_range(self, row):
        return pd.date_range(
                start = pd.to_datetime(f"{row['fecha_de_inicio'].year}-{row['fecha_de_inicio'].month}-01", format="%Y-%m-%d"), 
                periods = row['numero_de_plazos'], 
                freq = "2MS"
                ) + pd.DateOffset(days=row['fecha_de_inicio'].day - 1)
    
    def create_bimonthly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "bimestral"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_bimonthly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def create_weekly_date_range(self, row):
        return pd.date_range(start = row['fecha_de_inicio'], periods = row['numero_de_plazos'], freq = "7D") + pd.DateOffset(days=row['dias_de_margen'])
    
    def create_weekly_installments(self, frequency_desc_column: str = "frecuencia_de_pago", frecuency_desc_value: str = "semanal"):
        df = self.df.loc[self.df[frequency_desc_column] == frecuency_desc_value].copy()
        if len(df) > 0:
            df_new_installments = self.empty_installments_df
            for _,row in df.iterrows():
                date_range_i_row = self.create_weekly_date_range(row)
                if len(df_new_installments) == 0:
                    df_new_installments = self.create_nthly_installments(row, date_range_i_row)
                else:
                    df_new_installments = (
                        pd.concat([
                            df_new_installments,
                            self.create_nthly_installments(row, date_range_i_row)
                        ],
                            ignore_index=True
                        )
                    ) 
            print("Rows generated successfully")
            return df_new_installments
        else:
            return None
        
    def pdconcat_wise(self, base_df: pd.DataFrame, concat_df: pd.DataFrame):
        if len(base_df) > 0:
            return pd.concat(
                        [
                            base_df,
                            concat_df
                        ],
                            ignore_index=True
                    )
        else:
            return concat_df

    def create_all_installments(self):
        df_installments_cum = self.empty_installments_df
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_monthly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_semimonthly_installments())
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_bimonthly_installments()) 
        df_installments_cum = self.pdconcat_wise(df_installments_cum, self.create_weekly_installments())
        return df_installments_cum

In [84]:
installments_creator = InstallmentsCreator(dfp)

In [85]:
installments_creator.df

,fecha_de_transaccion,frecuencia_de_pago,fecha_de_inicio,numero_de_plazos,dias_de_margen,monto_total_deuda,forma_de_pago,descripcion,categoria,realizado_por,comentarios
0,2024-04-28,mensual,2024-04-28,12,0,12000.0,Hey Credito,Descripcion de gim,Gym,David,Comentario de gym
1,2024-04-30,mensual,2024-04-30,12,0,120000.0,Hey Credito,Description impuestos,Impuestos,David,Comentario de Impuestos
2,2024-05-04,quincenal,2024-05-05,12,2,146.0,Crédito Nomina,Credito nomina afirme,Credito Nomina,David,Comentario nomina
3,2024-05-04,bimestral,2024-05-14,12,0,5000.0,Hey Credito,Recibo Gas,Servicios,David,NaN
4,2024-05-04,semanal,2024-05-14,24,0,10000.0,LikeU,NaN,Arrendamiento,David,NaN
5,2024-05-04,mensual,2024-05-14,12,0,13000.0,Liverpool,Laptop,Arrendamiento,Wendy,NaN
6,2024-05-04,mensual,2024-05-14,12,0,13000.0,Liverpool,Laptop,Arrendamiento,David,Registro por deuda de wendy


In [86]:
df_all_installments = installments_creator.create_all_installments()
display(df_all_installments)

Rows generated successfully
Rows generated successfully
Rows generated successfully
Rows generated successfully


,fecha_de_cargo,descripcion,categoria,monto,forma_de_pago,realizado_por,limite_de_pago,deuda_a_plazos_recurrente_o_normal,comentarios
0,2024-04-28,Descripcion de gim,Gym,1000.000000,Hey Credito,David,2024-05-31,Plazos,Comentario de gym
1,2024-05-28,Descripcion de gim,Gym,1000.000000,Hey Credito,David,2024-07-01,Plazos,Comentario de gym
2,2024-06-28,Descripcion de gim,Gym,1000.000000,Hey Credito,David,2024-07-31,Plazos,Comentario de gym
3,2024-07-28,Descripcion de gim,Gym,1000.000000,Hey Credito,David,2024-08-31,Plazos,Comentario de gym
4,2024-08-28,Descripcion de gim,Gym,1000.000000,Hey Credito,David,2024-10-01,Plazos,Comentario de gym
...,...,...,...,...,...,...,...,...,...
91,2024-09-24,,Arrendamiento,416.666667,LikeU,David,2024-10-31,Plazos,
92,2024-10-01,,Arrendamiento,416.666667,LikeU,David,2024-10-31,Plazos,
93,2024-10-08,,Arrendamiento,416.666667,LikeU,David,2024-10-31,Plazos,
94,2024-10-15,,Arrendamiento,416.666667,LikeU,David,2024-12-01,Plazos,


In [64]:
df_new_semimonthly_installments = installments_creator.create_semimonthly_installments()
display(df_new_semimonthly_installments)

Rows generated successfully


,fecha_de_cargo,descripcion,categoria,monto,forma_de_pago,realizado_por,limite_de_pago,deuda_a_plazos_recurrente_o_normal,comentarios
0,2024-05-17,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-05-17,Plazos,Comentario nomina
1,2024-06-02,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-06-02,Plazos,Comentario nomina
2,2024-06-17,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-06-17,Plazos,Comentario nomina
3,2024-07-02,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-07-02,Plazos,Comentario nomina
4,2024-07-17,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-07-17,Plazos,Comentario nomina
5,2024-08-02,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-08-02,Plazos,Comentario nomina
6,2024-08-17,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-08-17,Plazos,Comentario nomina
7,2024-09-02,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-09-02,Plazos,Comentario nomina
8,2024-09-17,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-09-17,Plazos,Comentario nomina
9,2024-10-02,Credito nomina afirme,Credito Nomina,12.166667,Crédito Nomina,David,2024-10-02,Plazos,Comentario nomina


In [65]:
df_new_bimonthly_installments = installments_creator.create_bimonthly_installments() 
display(df_new_bimonthly_installments)

Rows generated successfully


,fecha_de_cargo,descripcion,categoria,monto,forma_de_pago,realizado_por,limite_de_pago,deuda_a_plazos_recurrente_o_normal,comentarios
0,2024-05-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2024-07-01,Plazos,
1,2024-07-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2024-08-31,Plazos,
2,2024-09-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2024-10-31,Plazos,
3,2024-11-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2024-12-31,Plazos,
4,2025-01-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2025-03-03,Plazos,
5,2025-03-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2025-05-01,Plazos,
6,2025-05-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2025-07-01,Plazos,
7,2025-07-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2025-08-31,Plazos,
8,2025-09-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2025-10-31,Plazos,
9,2025-11-14,Recibo Gas,Servicios,416.666667,Hey Credito,David,2025-12-31,Plazos,


In [66]:
df_new_weekly_installments = installments_creator.create_weekly_installments() 
display(df_new_weekly_installments)

Rows generated successfully


,fecha_de_cargo,descripcion,categoria,monto,forma_de_pago,realizado_por,limite_de_pago,deuda_a_plazos_recurrente_o_normal,comentarios
0,2024-05-14,,Arrendamiento,416.666667,LikeU,David,2024-07-01,Plazos,
1,2024-05-21,,Arrendamiento,416.666667,LikeU,David,2024-07-01,Plazos,
2,2024-05-28,,Arrendamiento,416.666667,LikeU,David,2024-07-01,Plazos,
3,2024-06-04,,Arrendamiento,416.666667,LikeU,David,2024-07-01,Plazos,
4,2024-06-11,,Arrendamiento,416.666667,LikeU,David,2024-07-01,Plazos,
5,2024-06-18,,Arrendamiento,416.666667,LikeU,David,2024-07-31,Plazos,
6,2024-06-25,,Arrendamiento,416.666667,LikeU,David,2024-07-31,Plazos,
7,2024-07-02,,Arrendamiento,416.666667,LikeU,David,2024-07-31,Plazos,
8,2024-07-09,,Arrendamiento,416.666667,LikeU,David,2024-07-31,Plazos,
9,2024-07-16,,Arrendamiento,416.666667,LikeU,David,2024-08-31,Plazos,
